In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime, timedelta

# Define a function to scrape the data for each season and output to CSV
def scrape_ncaa_womens_hockey(season_start, season_end):
    base_url = "https://www.uscho.com/scoreboard/division-i-women/{}/list-{}"
    all_games = []

    # Loop through each season and scrape data week by week
    for year in range(season_start, season_end + 1):
        for week in range(1, 30):  # Adjust number of weeks if necessary
            # Construct the URL for the specific week and season (list-1, list-2, etc.)
            url = base_url.format(f"{year}-{year + 1}", week)

            # Send request and parse HTML
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Print progress for each week
            print(f"Scraping {year}-{year + 1} - Week {week}...")

            # Find the table with game data
            rows = soup.find_all('tr')

            # Process the rows to extract relevant data
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 7:  # Ensure there are enough columns
                    try:
                        # Extract data (time, opponent, home score, away score)
                        time = cols[0].text.strip()
                        opponent = cols[1].text.strip()
                        home_score = cols[4].text.strip()
                        away_score = cols[5].text.strip()
                        location = cols[6].text.strip()

                        # Store the game data
                        all_games.append([year, week, time, opponent, home_score, away_score, location])
                    except IndexError:
                        # Handle rows that might be missing data
                        print(f"Skipping malformed row: {row}")

        # Convert list of games to DataFrame and save as CSV
        df = pd.DataFrame(all_games, columns=["Year", "Week", "Time", "Opponent", "Home Score", "Away Score", "Location"])
        df.to_csv(f"ncaa_womens_hockey_{year}_{year + 1}.csv", index=False)

    print("Scraping completed!")

# Scrape data from 2014-2015 to 2024-2025
scrape_ncaa_womens_hockey(2014, 2025)



Scraping 2014-2015 - Week 1...
Scraping 2014-2015 - Week 2...
Scraping 2014-2015 - Week 3...
Scraping 2014-2015 - Week 4...


KeyboardInterrupt: 

In [6]:
from google.colab import files

# Upload the file from your local machine
uploaded = files.upload()


MessageError: RangeError: Maximum call stack size exceeded.

In [7]:
import pandas as pd

# The file path where the uploaded file is saved
file_path = '/content/ncaa_womens_hockey_2014_2025.csv'

# Load the CSV data
data = pd.read_csv(file_path)

# Display the first few rows to check
print(data.head())


FileNotFoundError: [Errno 2] No such file or directory: '/content/ncaa_womens_hockey_2014_2025.csv'

In [ ]:
# Get all unique team names from the 'Home Team' and 'Opponent' columns
all_teams = pd.concat([data['Home Team'], data['Opponent']]).unique()

# Sort them for easier manual inspection
all_teams_sorted = sorted(all_teams)

print("All unique teams in dataset:")
for team in all_teams_sorted:
    print(team)



NameError: name 'data' is not defined

In [ ]:
TEAM_NAME_MAP = {
    "St. Anselm": "Saint Anselm",
    "St. Michael's": "Saint Michael's"
    # Add more if you see repeated alternate names for the same DI program
}
def canonical_name(team_name: str) -> str:
    """
    Return the standardized (canonical) team name.
    """
    team_name = team_name.strip()  # Remove extra whitespace
    return TEAM_NAME_MAP.get(team_name, team_name)
D1_TEAMS = {
    "Lindenwood",
    "Mercyhurst Lakers",
    "Penn State",
    "Robert Morris",
    "RIT",
    "Syracuse",
    "Brown",
    "Clarkson",
    "Colgate",
    "Cornell",
    "Dartmouth",
    "Harvard",
    "Princeton",
    "Quinnipiac",
    "Rensselaer",
    "St. Lawrence",
    "Union",
    "Yale",
    "Boston College",
    "Boston University",
    "Connecticut",
    "Holy Cross",
    "Maine",
    "Merrimack",
    "New Hampshire",
    "Northeastern",
    "Providence",
    "Vermont",
    "Assumption",
    "Franklin Pierce",
    "LIU",
    "Post",
    "Sacred Heart",
    "Saint Anselm",
    "Saint Michael's",
    "Stonehill",
    "Bemidji State",
    "Minnesota",
    "Minnesota Duluth",
    "Minnesota State",
    "Ohio State",
    "St. Cloud State",
    "St. Thomas",
    "Wisconsin"
}


In [ ]:
import pandas as pd

# Load your data
file_path = "ncaa_womens_hockey_2014_2025.csv"
data = pd.read_csv(file_path)

# Standardize columns
data['Home Team'] = data['Home Team'].apply(canonical_name)
data['Opponent'] = data['Opponent'].apply(canonical_name)

# Check the unique names again after mapping
all_teams_after = pd.concat([data['Home Team'], data['Opponent']]).unique()
print("Unique teams (after standardizing):")
print(sorted(all_teams_after))


Unique teams (after standardizing):
['Adrian', 'Alvernia', 'Amherst', 'Anna Maria', 'Assumption', 'Barrie', 'Becker', 'Bemidji State', 'Bluewater Jr Hawks', 'Boston Blades', 'Boston College', 'Boston Pride', 'Boston University', 'Bowdoin', 'British Columbia', 'Brock', 'Brown', 'Buffalo Beauts', 'Buffalo State', 'Cambridge', 'Cambridge Rivulettes', 'Canton', 'Carleton', 'Carleton (ON)', 'Castleton', 'China National Tm', 'Clarkson', 'Colby', 'Colgate', 'Concordia (PQ)', 'Concordia (QC)', 'Concordia (Que.)', 'Conn. College', 'Connecticut', 'Connecticut Whale', 'Cornell', 'Dartmouth', 'Durham', 'Durham W Lighting', 'Durham W Lightning', 'Durham West', 'Durham West Jr', 'Endicott', 'Franklin Pierce', 'Guelph', 'HTI', 'HTI Stars', 'Hamilton', 'Hamline', 'Harvard', 'Holy Cross', 'Johnson & Wales', "King's", 'Kingston (PWHL)', 'Korea Natl Team', 'Kunlun Red Star', 'LIU', 'Lake Forest', 'Laurentian', 'Lebanon Valley', 'Linden-Belleville', 'Lindenwood', 'Maine', 'Manhattanville', 'Manitoba', 'Ma

In [ ]:
import math

initial_rating = 1500

def update_elo(rating_A, rating_B, score_A, score_B, K=30):
    # Standard Elo with margin-of-victory factor
    E_A = 1 / (1 + 10 ** ((rating_B - rating_A) / 400))
    E_B = 1 / (1 + 10 ** ((rating_A - rating_B) / 400))

    # Determine actual outcome
    if score_A > score_B:
        S_A, S_B = 1, 0
    elif score_A < score_B:
        S_A, S_B = 0, 1
    else:
        S_A, S_B = 0.5, 0.5

    # Margin of victory multiplier
    mov = abs(score_A - score_B)
    if mov == 0:
        mov_multiplier = 1.0
    else:
        mov_multiplier = math.log(mov + 1) * (2.2 / (1 + 0.001 * abs(rating_A - rating_B)))

    new_rating_A = rating_A + K * mov_multiplier * (S_A - E_A)
    new_rating_B = rating_B + K * mov_multiplier * (S_B - E_B)

    return new_rating_A, new_rating_B

def normalize_end_of_season(team_ratings, initial_rating=1500, factor=0.25):
    """
    Bring ratings 25% back to the mean at the end of each season.
    Only done for DI teams.
    """
    for team in team_ratings:
        r = team_ratings[team]
        if r > initial_rating:
            team_ratings[team] = r - (r - initial_rating) * factor
        else:
            team_ratings[team] = r + (initial_rating - r) * factor

# ------------------------------------------------

# Clean your data, sort, etc.
data_cleaned = data.dropna(subset=['Home Score', 'Opp Score'])
data_cleaned = data_cleaned.sort_values(by=['Year', 'Week', 'Home Team', 'Opponent'])

team_ratings = {}         # Ratings for DI teams
temp_non_di_ratings = {}  # Ratings for non-DI teams
all_ratings = []
previous_year = None

def get_team_rating(team_name):
    """
    Return a team's current rating.
    Store DI teams in team_ratings, non-DI teams in temp_non_di_ratings.
    """
    if team_name in D1_TEAMS:
        if team_name not in team_ratings:
            team_ratings[team_name] = initial_rating
        return team_ratings[team_name]
    else:
        # Non-DI
        if team_name not in temp_non_di_ratings:
            temp_non_di_ratings[team_name] = initial_rating
        return temp_non_di_ratings[team_name]

def set_team_rating(team_name, new_rating):
    """
    Update the rating in the correct dictionary.
    """
    if team_name in D1_TEAMS:
        team_ratings[team_name] = new_rating
    else:
        temp_non_di_ratings[team_name] = new_rating

for idx, row in data_cleaned.iterrows():
    current_year = row['Year']

    # End-of-year regression if needed
    if previous_year is not None and current_year != previous_year:
        normalize_end_of_season(team_ratings, initial_rating=initial_rating, factor=0.25)

    previous_year = current_year

    home_team = row['Home Team']
    away_team = row['Opponent']
    home_score = int(row['Home Score'])
    away_score = int(row['Opp Score'])

    # Get current ratings
    rating_home = get_team_rating(home_team)
    rating_away = get_team_rating(away_team)

    # Update Elo
    new_rating_home, new_rating_away = update_elo(rating_home, rating_away,
                                                  home_score, away_score, K=30)

    set_team_rating(home_team, new_rating_home)
    set_team_rating(away_team, new_rating_away)

    # Record the result (optionally)
    all_ratings.append([
        current_year,
        row['Week'],
        home_team,
        away_team,
        home_score,
        away_score,
        new_rating_home,
        new_rating_away
    ])

# After final season
normalize_end_of_season(team_ratings, initial_rating=initial_rating, factor=0.25)

# Convert to DataFrame
elo_df = pd.DataFrame(all_ratings, columns=[
    "Year", "Week", "Home Team", "Away Team",
    "Home Score", "Away Score", "Home Team ELO", "Away Team ELO"
])

# If you want to EXCLUDE rows that are non-DI vs non-DI from the final CSV:
# (i.e., only show games where at least one team is DI)
def at_least_one_DI(row):
    return (row["Home Team"] in D1_TEAMS) or (row["Away Team"] in D1_TEAMS)

elo_df = elo_df[elo_df.apply(at_least_one_DI, axis=1)]

# Save final
elo_df.to_csv("ncaa_womens_hockey_elo_ratings_DI_only.csv", index=False)
print(elo_df.head(15))



    Year  Week          Home Team           Away Team  Home Score  Away Score  \
0   2014     1         Quinnipiac   China National Tm           8           0   
1   2014     1      Robert Morris  Bluewater Jr Hawks          10           2   
2   2014     1       St. Lawrence              McGill           4           7   
3   2014     2     Boston College             Queen's          10           1   
4   2014     2  Boston University             Queen's           3           0   
5   2014     2            Colgate           Cambridge           3           0   
6   2014     2        Connecticut    Minnesota Duluth           4           4   
7   2014     2        Connecticut    Minnesota Duluth           0           3   
8   2014     2         Lindenwood           Wisconsin           1           5   
9   2014     2         Lindenwood           Wisconsin           0           6   
10  2014     2              Maine       New Hampshire           2           1   
13  2014     2       Northea

Mounted at /content/drive


In [ ]:
# Display the entire DataFrame
print(elo_df)


      Year  Week          Home Team           Away Team  Home Score  \
0     2014     1         Quinnipiac   China National Tm           8   
1     2014     1      Robert Morris  Bluewater Jr Hawks          10   
2     2014     1       St. Lawrence              McGill           4   
3     2014     2     Boston College             Queen's          10   
4     2014     2  Boston University             Queen's           3   
...    ...   ...                ...                 ...         ...   
7561  2024    19              Union          Rensselaer           2   
7562  2024    19            Vermont         Connecticut           2   
7563  2024    19            Vermont         Connecticut           0   
7564  2024    19          Wisconsin     St. Cloud State           5   
7565  2024    19          Wisconsin     St. Cloud State           2   

      Away Score  Home Team ELO  Away Team ELO  
0              0    1572.508411    1427.491589  
1              2    1572.508411    1427.491589  


In [ ]:
from google.colab import files

# Save the ELO ratings DataFrame to a CSV file
elo_df.to_csv('ncaa_womens_hockey_elo_ratings_2014_2025.csv', index=False)

# Download the CSV file
files.download('ncaa_womens_hockey_elo_ratings_2014_2025.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Strip any leading/trailing spaces from column names
data.columns = data.columns.str.strip()

# Check the column names again after cleaning
print(data.columns)


Index(['Year', 'Week', 'Time', 'Opponent', 'Opp Score', 'Home Team',
       'Home Score', 'OT', 'Notes', 'Type', 'Summary Link', 'Location'],
      dtype='object')


In [ ]:
from google.colab import files

# Upload the file from your local machine
uploaded = files.upload()


Saving ncaa_womens_hockey_elo_ratings_2014_2025.csv to ncaa_womens_hockey_elo_ratings_2014_2025.csv


In [ ]:
# Load the two datasets
import pandas as pd

# Load the first dataset (game results)
game_results_path = 'ncaa_womens_hockey_2014_2025.csv'
game_results = pd.read_csv(game_results_path)

# Load the second dataset (ELO ratings)
elo_ratings_path = 'ncaa_womens_hockey_elo_ratings_2014_2025.csv'
elo_ratings = pd.read_csv(elo_ratings_path)

# Display the first few rows of each to inspect their structure
print("Game Results:")
print(game_results.head())

print("\nELO Ratings:")
print(elo_ratings.head())


Game Results:
   Year  Week     Time            Opponent  Opp Score      Home Team  \
0  2014     1  2:00 ET   China National Tm        0.0     Quinnipiac   
1  2014     1  7:00 ET  Bluewater Jr Hawks        2.0  Robert Morris   
2  2014     1  5:00 ET              McGill        7.0   St. Lawrence   
3  2014     2  7:00 CT           Wisconsin        5.0     Lindenwood   
4  2014     2  7:00 ET       Bemidji State        5.0  Robert Morris   

   Home Score   OT Notes  Type  \
0         8.0  NaN   NaN   NaN   
1        10.0  NaN   NaN   NaN   
2         4.0  NaN   NaN   NaN   
3         1.0  NaN   NaN   NaN   
4         2.0  NaN   NaN   NaN   

                                        Summary Link  \
0  /gameday/division-i-women/2014-2015/2014-09-20...   
1  /gameday/division-i-women/2014-2015/2014-09-20...   
2  /gameday/division-i-women/2014-2015/2014-09-24...   
3  /gameday/division-i-women/2014-2015/2014-09-26...   
4  /gameday/division-i-women/2014-2015/2014-09-26...   

           

In [ ]:
# Rename the 'Opponent' column to 'Away Team' and 'Opp Score' to 'Away Score' in the game_results DataFrame
game_results.rename(columns={'Opponent': 'Away Team', 'Opp Score': 'Away Score'}, inplace=True)

# Verify the column names after renaming
print(game_results.columns)


Index(['Year', 'Week', 'Time', 'Away Team', 'Away Score', 'Home Team',
       'Home Score', 'OT', 'Notes', 'Type', 'Summary Link', 'Location'],
      dtype='object')


In [ ]:
# Merge the datasets on 'Year', 'Week', 'Home Team', 'Away Team'
final_merged_data = pd.merge(game_results, elo_ratings[['Year', 'Week', 'Home Team', 'Away Team', 'Home Team ELO', 'Away Team ELO']],
                             how='left', left_on=['Year', 'Week', 'Home Team', 'Away Team'],
                             right_on=['Year', 'Week', 'Home Team', 'Away Team'])

# Create a new DataFrame with the selected columns
final_df = final_merged_data[['Year', 'Week', 'Home Team', 'Away Team', 'Home Score', 'Away Score', 'Home Team ELO', 'Away Team ELO']]

# Display the first few rows of the merged DataFrame
print(final_df)


       Year  Week      Home Team           Away Team  Home Score  Away Score  \
0      2014     1     Quinnipiac   China National Tm         8.0         0.0   
1      2014     1  Robert Morris  Bluewater Jr Hawks        10.0         2.0   
2      2014     1   St. Lawrence              McGill         4.0         7.0   
3      2014     2     Lindenwood           Wisconsin         1.0         5.0   
4      2014     2     Lindenwood           Wisconsin         1.0         5.0   
...     ...   ...            ...                 ...         ...         ...   
11561  2024    23      Stonehill          Assumption         NaN         NaN   
11562  2024    23  Bemidji State           Wisconsin         NaN         NaN   
11563  2024    23     Ohio State     St. Cloud State         NaN         NaN   
11564  2024    23     St. Thomas     Minnesota State         NaN         NaN   
11565  2024    23      Minnesota    Minnesota Duluth         NaN         NaN   

       Home Team ELO  Away Team ELO  
0

In [ ]:
# Create the score differential column
final_df['Score Differential'] = final_df['Home Score'] - final_df['Away Score']

# Display the first few rows to verify the new column
print(final_df)


       Year  Week      Home Team           Away Team  Home Score  Away Score  \
0      2014     1     Quinnipiac   China National Tm         8.0         0.0   
1      2014     1  Robert Morris  Bluewater Jr Hawks        10.0         2.0   
2      2014     1   St. Lawrence              McGill         4.0         7.0   
3      2014     2     Lindenwood           Wisconsin         1.0         5.0   
4      2014     2     Lindenwood           Wisconsin         1.0         5.0   
...     ...   ...            ...                 ...         ...         ...   
11561  2024    23      Stonehill          Assumption         NaN         NaN   
11562  2024    23  Bemidji State           Wisconsin         NaN         NaN   
11563  2024    23     Ohio State     St. Cloud State         NaN         NaN   
11564  2024    23     St. Thomas     Minnesota State         NaN         NaN   
11565  2024    23      Minnesota    Minnesota Duluth         NaN         NaN   

       Home Team ELO  Away Team ELO  Sc

<ipython-input-14-71fd346dc88f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Score Differential'] = final_df['Home Score'] - final_df['Away Score']


In [ ]:
# Remove rows with NaN values for 2024 season
final_df = final_df[(final_df['Year'] != 2024) | (final_df['Home Score'].notna() & final_df['Away Score'].notna())]

# Display the first few rows to verify
print(final_df)


       Year  Week      Home Team           Away Team  Home Score  Away Score  \
0      2014     1     Quinnipiac   China National Tm         8.0         0.0   
1      2014     1  Robert Morris  Bluewater Jr Hawks        10.0         2.0   
2      2014     1   St. Lawrence              McGill         4.0         7.0   
3      2014     2     Lindenwood           Wisconsin         1.0         5.0   
4      2014     2     Lindenwood           Wisconsin         1.0         5.0   
...     ...   ...            ...                 ...         ...         ...   
11406  2024    19      Wisconsin     St. Cloud State         5.0         2.0   
11407  2024    19     Rensselaer               Union         2.0         3.0   
11408  2024    19   Sacred Heart                 LIU         0.0         3.0   
11409  2024    19      Wisconsin     St. Cloud State         2.0         2.0   
11410  2024    19      Wisconsin     St. Cloud State         2.0         2.0   

       Home Team ELO  Away Team ELO  Sc

In [ ]:
print(final_df)

       Year  Week      Home Team           Away Team  Home Score  Away Score  \
0      2014     1     Quinnipiac   China National Tm         8.0         0.0   
1      2014     1  Robert Morris  Bluewater Jr Hawks        10.0         2.0   
2      2014     1   St. Lawrence              McGill         4.0         7.0   
3      2014     2     Lindenwood           Wisconsin         1.0         5.0   
4      2014     2     Lindenwood           Wisconsin         1.0         5.0   
...     ...   ...            ...                 ...         ...         ...   
11406  2024    19      Wisconsin     St. Cloud State         5.0         2.0   
11407  2024    19     Rensselaer               Union         2.0         3.0   
11408  2024    19   Sacred Heart                 LIU         0.0         3.0   
11409  2024    19      Wisconsin     St. Cloud State         2.0         2.0   
11410  2024    19      Wisconsin     St. Cloud State         2.0         2.0   

       Home Team ELO  Away Team ELO  Sc

In [ ]:
# Sort teams by their final ELO rating in descending order
sorted_ratings = sorted(team_ratings.items(), key=lambda x: x[1], reverse=True)

print("Final ELO Ratings:")
for team, rating in sorted_ratings:
    print(f"{team}: {rating:.2f}")


Final ELO Ratings:
Wisconsin: 1984.21
Ohio State: 1885.44
Colgate: 1856.74
Cornell: 1813.06
Minnesota: 1793.42
Penn State: 1732.68
St. Lawrence: 1709.35
Minnesota Duluth: 1703.46
Princeton: 1697.54
Clarkson: 1695.04
Quinnipiac: 1681.36
Connecticut: 1651.48
Northeastern: 1649.88
Providence: 1644.36
Boston College: 1634.02
St. Cloud State: 1621.77
Yale: 1607.00
Minnesota State: 1594.90
Boston University: 1566.37
Brown: 1549.75
Maine: 1506.57
Union: 1495.52
Rensselaer: 1494.01
RIT: 1473.30
New Hampshire: 1468.96
Dartmouth: 1450.26
St. Thomas: 1449.23
Syracuse: 1429.94
LIU: 1419.30
Bemidji State: 1393.10
Stonehill: 1374.57
Vermont: 1370.64
Holy Cross: 1370.09
Robert Morris: 1362.07
Merrimack: 1356.50
Sacred Heart: 1337.25
Harvard: 1334.00
Franklin Pierce: 1321.41
Lindenwood: 1282.17
Saint Anselm: 1277.75
Post: 1274.09
Assumption: 1194.51
Saint Michael's: 1071.72


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Prepare the data for Logistic Regression
# Feature: ELO difference
X = final_df[['Home Team ELO', 'Away Team ELO']].copy()
X['ELO_diff'] = X['Home Team ELO'] - X['Away Team ELO']
X = X[['ELO_diff']]

# Target: 1 if Home Team wins, 0 if Away Team wins
final_df['Result'] = (final_df['Score Differential'] > 0).astype(int)
y = final_df['Result']

# Drop any NaN values
X = X.dropna()
y = y[X.index]

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Logistic Regression model
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

# Predict on the test set
y_pred = log_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Function to predict win probabilities based on team names
def predict_win_probabilities(home_team_name, away_team_name, df=final_df):
    # Look up ELO ratings
    home_team_elo = df[df['Home Team'] == home_team_name]['Home Team ELO'].values
    away_team_elo = df[df['Away Team'] == away_team_name]['Away Team ELO'].values

    # Check if both teams exist in the dataset
    if len(home_team_elo) == 0:
        return f"Home team '{home_team_name}' not found in the dataset."
    if len(away_team_elo) == 0:
        return f"Away team '{away_team_name}' not found in the dataset."

    # Extract the first ELO rating if multiple entries exist
    home_team_elo = home_team_elo[0]
    away_team_elo = away_team_elo[0]

    # Calculate ELO difference
    elo_diff = home_team_elo - away_team_elo

    # Predict win probabilities
    prob_home_win = log_model.predict_proba([[elo_diff]])[0][1]  # Probability of home team winning
    prob_away_win = 1 - prob_home_win  # Probability of away team winning

    return {
        'Home Team': home_team_name,
        'Away Team': away_team_name,
        'Home Win Probability': round(prob_home_win * 100, 2),
        'Away Win Probability': round(prob_away_win * 100, 2)
    }

# Example usage
home_team = "Dartmouth"
away_team = "Harvard"

result = predict_win_probabilities(home_team, away_team)

print(f"{result['Home Team']} Win Probability: {result['Home Win Probability']}%")
print(f"{result['Away Team']} Win Probability: {result['Away Win Probability']}%")


Model Accuracy: 75.84%
Dartmouth Win Probability: 44.27%
Harvard Win Probability: 55.73%


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Assuming your model has already been trained and 'final_df' is available

# Function to get Elo ratings for teams
def get_team_elo(home_team, away_team, df):
    home_elo = df[df['Home Team'] == home_team]['Home Team ELO'].values
    away_elo = df[df['Away Team'] == away_team]['Away Team ELO'].values

    if len(home_elo) == 0 or len(away_elo) == 0:
        return None, None

    return home_elo[0], away_elo[0]

# Function to predict the score differential between two teams
def predict_score_differential(home_team, away_team, model, df):
    # Get the Elo ratings for both teams
    home_elo, away_elo = get_team_elo(home_team, away_team, df)

    if home_elo is None or away_elo is None:
        return f"One or both teams not found in the Elo ratings."

    # Calculate the Elo difference
    elo_diff = home_elo - away_elo

    # Predict the probability of home team winning using the model
    home_win_prob = model.predict_proba([[elo_diff]])[0][1]  # Probability of home team winning
    away_win_prob = 1 - home_win_prob  # Probability of away team winning

    # Return predicted score differential (could use the Elo difference as a proxy)
    predicted_score_diff = elo_diff/20
    return {
        'Home Team': home_team,
        'Away Team': away_team,
        'Predicted Score Differential': predicted_score_diff,
        'Home Win Probability': round(home_win_prob * 100, 2),
        'Away Win Probability': round(away_win_prob * 100, 2)
    }

# Example usage
home_team = "Dartmouth"
away_team = "Harvard"

result = predict_score_differential(home_team, away_team, log_model, final_df)

# Output the result
print(f"Predicted Score Differential: {result['Predicted Score Differential']:.2f}")
print(f"{result['Home Team']} Win Probability: {result['Home Win Probability']}%")
print(f"{result['Away Team']} Win Probability: {result['Away Win Probability']}%")


Predicted Score Differential: -1.28
Dartmouth Win Probability: 44.27%
Harvard Win Probability: 55.73%


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
